In [3]:
from LorisBallsBasedModel import *
import tensorflow as tf
import pandas as pd
import numpy as np

In [2]:
cols = ['price', 'maint', 'doors', 'persons', 'lug_capacity', 'safety','output']
cars = pd.read_csv(r'data\car_evaluation_dataset\car_evaluation.csv', names=cols, header=None)
cars.head(3)

,price,maint,doors,persons,lug_capacity,safety,output
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc


In [3]:
# one-hot encoding
price = pd.get_dummies(cars.price, prefix='price')
maint = pd.get_dummies(cars.maint, prefix='maint')
doors = pd.get_dummies(cars.doors, prefix='doors')
persons = pd.get_dummies(cars.persons, prefix='persons')
lug_capacity = pd.get_dummies(cars.lug_capacity, prefix='lug_capacity')
safety = pd.get_dummies(cars.safety, prefix='safety')

labels = pd.get_dummies(cars.output, prefix='condition')

X = pd.concat([price, maint, doors, persons, lug_capacity, safety] , axis=1)
y = labels.values

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

# SingleLayerPerceptron base model

In [10]:
SLP = SingleLayerPerceptron(tf.keras.layers.Dense(4, activation='softmax'))
SLP.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=['acc'])

In [11]:
history = SLP.fit(X_train, y_train, batch_size=8, epochs=50, verbose=1, validation_split=0.2)

Epoch 1/50
139/139 [==============================] - 1s 2ms/step - loss: 1.2879 - acc: 0.4706 - val_loss: 1.1141 - val_acc: 0.6282
Epoch 2/50
139/139 [==============================] - 0s 1ms/step - loss: 0.9272 - acc: 0.7005 - val_loss: 0.9357 - val_acc: 0.6534
Epoch 3/50
139/139 [==============================] - 0s 1ms/step - loss: 0.7847 - acc: 0.7195 - val_loss: 0.8499 - val_acc: 0.6498
Epoch 4/50
139/139 [==============================] - 0s 1ms/step - loss: 0.7068 - acc: 0.7195 - val_loss: 0.7899 - val_acc: 0.6498
Epoch 5/50
139/139 [==============================] - 0s 1ms/step - loss: 0.6518 - acc: 0.7231 - val_loss: 0.7439 - val_acc: 0.6570
Epoch 6/50
139/139 [==============================] - 0s 1ms/step - loss: 0.6097 - acc: 0.7285 - val_loss: 0.7053 - val_acc: 0.6643
Epoch 7/50
139/139 [==============================] - 0s 1ms/step - loss: 0.5751 - acc: 0.7430 - val_loss: 0.6732 - val_acc: 0.6715
Epoch 8/50
139/139 [==============================] - 0s 1ms/step - loss: 0.

In [12]:
SLP.evaluate(X_test, y_test)

11/11 [==============================] - 0s 2ms/step - loss: 0.2699 - acc: 0.8873


[0.26990699768066406, 0.8872832655906677]

# Transfert learning
## SemiAxisRegularizer effect

In [13]:
LBBM_1 = LorisBallsBasedModel(BoundedParaboloids(128, semi_axis_regularizer=SemiAxisRegularizer(0.000001)),
                              tf.keras.layers.Dense(4, activation='softmax'),
                              SLP)
LBBM_1.freeze_base_model()
LBBM_1.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=['acc'])

In [14]:
history_LBBM_1 = LBBM_1.fit(X_train, y_train, batch_size=8, epochs=50, verbose=1, validation_split=0.2)

Epoch 1/50
139/139 [==============================] - 48s 64ms/step - loss: 0.8881 - acc: 0.6950 - val_loss: 0.9731 - val_acc: 0.6498
Epoch 2/50
139/139 [==============================] - 5s 34ms/step - loss: 0.7802 - acc: 0.7195 - val_loss: 0.8744 - val_acc: 0.6498
Epoch 3/50
139/139 [==============================] - 4s 29ms/step - loss: 0.7074 - acc: 0.7195 - val_loss: 0.8195 - val_acc: 0.6498
Epoch 4/50
139/139 [==============================] - 3s 24ms/step - loss: 0.6455 - acc: 0.7195 - val_loss: 0.7304 - val_acc: 0.6498
Epoch 5/50
139/139 [==============================] - 3s 23ms/step - loss: 0.5820 - acc: 0.7593 - val_loss: 0.6468 - val_acc: 0.7906
Epoch 6/50
139/139 [==============================] - 3s 23ms/step - loss: 0.5401 - acc: 0.7991 - val_loss: 0.6294 - val_acc: 0.7040
Epoch 7/50
139/139 [==============================] - 3s 22ms/step - loss: 0.4891 - acc: 0.8353 - val_loss: 0.5837 - val_acc: 0.8267
Epoch 8/50
139/139 [==============================] - 3s 22ms/step -

In [15]:
LBBM_1.evaluate(X_test, y_test)

11/11 [==============================] - 0s 8ms/step - loss: 0.1413 - acc: 0.9595


[0.1412847638130188, 0.9595375657081604]

In [16]:
LBBM_1.base_model.evaluate(X_test, y_test)

11/11 [==============================] - 0s 1ms/step - loss: 0.2699 - acc: 0.8873


[0.26990699768066406, 0.8872832655906677]

In [17]:
LBBM_2 = LorisBallsBasedModel(BoundedParaboloids(128, semi_axis_regularizer=SemiAxisRegularizer(0.00001)),
                              tf.keras.layers.Dense(4, activation='softmax'),
                              SLP)
LBBM_2.freeze_base_model()
LBBM_2.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=['acc'])

In [18]:
history_LBBM_2 = LBBM_2.fit(X_train, y_train, batch_size=8, epochs=50, verbose=1, validation_split=0.2)

Epoch 1/50
139/139 [==============================] - 42s 53ms/step - loss: 0.9990 - acc: 0.7195 - val_loss: 1.0305 - val_acc: 0.6498
Epoch 2/50
139/139 [==============================] - 3s 19ms/step - loss: 0.8503 - acc: 0.7195 - val_loss: 0.9358 - val_acc: 0.6498
Epoch 3/50
139/139 [==============================] - 3s 19ms/step - loss: 0.7754 - acc: 0.7195 - val_loss: 0.8704 - val_acc: 0.6498
Epoch 4/50
139/139 [==============================] - 3s 21ms/step - loss: 0.7191 - acc: 0.7204 - val_loss: 0.8098 - val_acc: 0.6534
Epoch 5/50
139/139 [==============================] - 3s 21ms/step - loss: 0.6687 - acc: 0.7385 - val_loss: 0.7609 - val_acc: 0.6570
Epoch 6/50
139/139 [==============================] - 3s 20ms/step - loss: 0.6170 - acc: 0.7656 - val_loss: 0.7172 - val_acc: 0.7401
Epoch 7/50
139/139 [==============================] - 3s 20ms/step - loss: 0.5693 - acc: 0.8226 - val_loss: 0.6721 - val_acc: 0.7076
Epoch 8/50
139/139 [==============================] - 3s 19ms/step -

In [19]:
LBBM_2.evaluate(X_test, y_test)

11/11 [==============================] - 0s 8ms/step - loss: 0.1634 - acc: 0.9451


[0.1634235978126526, 0.9450867176055908]

In [20]:
LBBM_2.base_model.evaluate(X_test, y_test)

11/11 [==============================] - 0s 2ms/step - loss: 0.2699 - acc: 0.8873


[0.26990699768066406, 0.8872832655906677]

In [21]:
LBBM_3 = LorisBallsBasedModel(BoundedParaboloids(128, semi_axis_regularizer=SemiAxisRegularizer(0.00005)),
                              tf.keras.layers.Dense(4, activation='softmax'),
                              SLP)
LBBM_3.freeze_base_model()
LBBM_3.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=['acc'])

In [22]:
history_LBBM_3 = LBBM_3.fit(X_train, y_train, batch_size=8, epochs=50, verbose=1, validation_split=0.2)

Epoch 1/50
139/139 [==============================] - 46s 58ms/step - loss: 1.9178 - acc: 0.6561 - val_loss: 1.6670 - val_acc: 0.6498
Epoch 2/50
139/139 [==============================] - 3s 19ms/step - loss: 1.3782 - acc: 0.7195 - val_loss: 1.4289 - val_acc: 0.6498
Epoch 3/50
139/139 [==============================] - 3s 19ms/step - loss: 1.2041 - acc: 0.7195 - val_loss: 1.2668 - val_acc: 0.6498
Epoch 4/50
139/139 [==============================] - 3s 20ms/step - loss: 1.0877 - acc: 0.7195 - val_loss: 1.1756 - val_acc: 0.6498
Epoch 5/50
139/139 [==============================] - 3s 20ms/step - loss: 1.0031 - acc: 0.7231 - val_loss: 1.0868 - val_acc: 0.6498
Epoch 6/50
139/139 [==============================] - 3s 20ms/step - loss: 0.9307 - acc: 0.7330 - val_loss: 1.0502 - val_acc: 0.6498
Epoch 7/50
139/139 [==============================] - 3s 20ms/step - loss: 0.8687 - acc: 0.7448 - val_loss: 0.9578 - val_acc: 0.7906
Epoch 8/50
139/139 [==============================] - 3s 20ms/step -

In [24]:
LBBM_3.evaluate(X_test, y_test)

11/11 [==============================] - 0s 7ms/step - loss: 0.2450 - acc: 0.9422


[0.24502310156822205, 0.9421965479850769]

In [25]:
LBBM_4 = LorisBallsBasedModel(BoundedParaboloids(128, semi_axis_regularizer=SemiAxisRegularizer(0.0001)),
                              tf.keras.layers.Dense(4, activation='softmax'),
                              SLP)
LBBM_4.freeze_base_model()
LBBM_4.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=['acc'])

In [26]:
history_LBBM_4 = LBBM_4.fit(X_train, y_train, batch_size=8, epochs=50, verbose=1, validation_split=0.2)

Epoch 1/50
139/139 [==============================] - 46s 66ms/step - loss: 2.8767 - acc: 0.6914 - val_loss: 2.3827 - val_acc: 0.6498
Epoch 2/50
139/139 [==============================] - 3s 22ms/step - loss: 2.0113 - acc: 0.7195 - val_loss: 1.9832 - val_acc: 0.6498
Epoch 3/50
139/139 [==============================] - 3s 23ms/step - loss: 1.7126 - acc: 0.7195 - val_loss: 1.7298 - val_acc: 0.6498
Epoch 4/50
139/139 [==============================] - 3s 22ms/step - loss: 1.5180 - acc: 0.7195 - val_loss: 1.5678 - val_acc: 0.6498
Epoch 5/50
139/139 [==============================] - 3s 23ms/step - loss: 1.3816 - acc: 0.7222 - val_loss: 1.4675 - val_acc: 0.6498
Epoch 6/50
139/139 [==============================] - 3s 23ms/step - loss: 1.2702 - acc: 0.7367 - val_loss: 1.3614 - val_acc: 0.6498
Epoch 7/50
139/139 [==============================] - 3s 21ms/step - loss: 1.1823 - acc: 0.7602 - val_loss: 1.2742 - val_acc: 0.6498
Epoch 8/50
139/139 [==============================] - 3s 20ms/step -

In [27]:
LBBM_4.evaluate(X_test, y_test)

11/11 [==============================] - 0s 9ms/step - loss: 0.3490 - acc: 0.9191


[0.3489813208580017, 0.9190751314163208]

=> SemiAxisRegularizer need to be small for this example (~ 0.000001-0.00001).

## Retraining base model (after doing transfert learning):

In [28]:
LBBM = LorisBallsBasedModel(BoundedParaboloids(128, semi_axis_regularizer=SemiAxisRegularizer(0.00001)),
                            tf.keras.layers.Dense(4, activation='softmax'),
                            SLP)
LBBM.freeze_base_model()
LBBM.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=['acc'])

In [29]:
history_LBBM = LBBM.fit(X_train, y_train, batch_size=8, epochs=50, verbose=1, validation_split=0.2)

Epoch 1/50
139/139 [==============================] - 41s 60ms/step - loss: 0.9622 - acc: 0.7195 - val_loss: 0.9969 - val_acc: 0.6498
Epoch 2/50
139/139 [==============================] - 3s 19ms/step - loss: 0.8284 - acc: 0.7195 - val_loss: 0.9182 - val_acc: 0.6498
Epoch 3/50
139/139 [==============================] - 3s 20ms/step - loss: 0.7550 - acc: 0.7195 - val_loss: 0.8647 - val_acc: 0.6498
Epoch 4/50
139/139 [==============================] - 3s 20ms/step - loss: 0.6998 - acc: 0.7276 - val_loss: 0.8169 - val_acc: 0.6498
Epoch 5/50
139/139 [==============================] - 3s 20ms/step - loss: 0.6504 - acc: 0.7439 - val_loss: 0.7453 - val_acc: 0.6859
Epoch 6/50
139/139 [==============================] - 3s 20ms/step - loss: 0.5985 - acc: 0.7864 - val_loss: 0.6957 - val_acc: 0.7617
Epoch 7/50
139/139 [==============================] - 3s 20ms/step - loss: 0.5562 - acc: 0.8308 - val_loss: 0.6435 - val_acc: 0.8014
Epoch 8/50
139/139 [==============================] - 3s 20ms/step -

In [30]:
LBBM.evaluate(X_test, y_test)

11/11 [==============================] - 0s 8ms/step - loss: 0.1426 - acc: 0.9451


[0.14263127744197845, 0.9450867176055908]

In [31]:
LBBM.base_model.evaluate(X_test, y_test)

11/11 [==============================] - 0s 2ms/step - loss: 0.2699 - acc: 0.8873


[0.26990699768066406, 0.8872832655906677]

In [32]:
LBBM.unfreeze_model()
LBBM.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005), metrics=['acc'])

In [33]:
history_LBBM = LBBM.fit(X_train, y_train, batch_size=8, epochs=50, verbose=1, validation_split=0.2)

Epoch 1/50
139/139 [==============================] - 47s 62ms/step - loss: 0.0916 - acc: 0.9846 - val_loss: 0.1386 - val_acc: 0.9639
Epoch 2/50
139/139 [==============================] - 3s 22ms/step - loss: 0.0906 - acc: 0.9846 - val_loss: 0.1402 - val_acc: 0.9531
Epoch 3/50
139/139 [==============================] - 3s 22ms/step - loss: 0.0865 - acc: 0.9910 - val_loss: 0.1365 - val_acc: 0.9531
Epoch 4/50
139/139 [==============================] - 3s 22ms/step - loss: 0.0855 - acc: 0.9900 - val_loss: 0.1316 - val_acc: 0.9675
Epoch 5/50
139/139 [==============================] - 3s 23ms/step - loss: 0.0830 - acc: 0.9900 - val_loss: 0.1306 - val_acc: 0.9711
Epoch 6/50
139/139 [==============================] - 3s 22ms/step - loss: 0.0816 - acc: 0.9910 - val_loss: 0.1275 - val_acc: 0.9711
Epoch 7/50
139/139 [==============================] - 3s 23ms/step - loss: 0.0797 - acc: 0.9900 - val_loss: 0.1359 - val_acc: 0.9495
Epoch 8/50
139/139 [==============================] - 3s 23ms/step -

In [34]:
LBBM.evaluate(X_test, y_test)

11/11 [==============================] - 0s 9ms/step - loss: 0.0810 - acc: 0.9769


[0.08101316541433334, 0.9768785834312439]

In [35]:
LBBM.base_model.evaluate(X_test, y_test)

11/11 [==============================] - 0s 2ms/step - loss: 0.2627 - acc: 0.8786


[0.2626914083957672, 0.8786126971244812]

=> The test accuracy is the best one untill now!

## Adding a second BoundedParaboloids layer

In [36]:
LBBM_2_layers = LorisBallsBasedModel(BoundedParaboloids(128, semi_axis_regularizer=SemiAxisRegularizer(0.00001)),
                                     tf.keras.layers.Dense(4, activation='softmax'),
                                     LBBM)
LBBM_2_layers.freeze_base_model()
LBBM_2_layers.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=['acc'])

In [37]:
history_LBBM_2_layers = LBBM_2_layers.fit(X_train, y_train, batch_size=8, epochs=50, verbose=1, validation_split=0.2)

Epoch 1/50
139/139 [==============================] - 73s 116ms/step - loss: 1.0301 - acc: 0.6525 - val_loss: 0.9539 - val_acc: 0.6498
Epoch 2/50
139/139 [==============================] - 4s 28ms/step - loss: 0.7671 - acc: 0.7195 - val_loss: 0.8560 - val_acc: 0.6498
Epoch 3/50
139/139 [==============================] - 4s 27ms/step - loss: 0.6781 - acc: 0.7357 - val_loss: 0.7446 - val_acc: 0.8628
Epoch 4/50
139/139 [==============================] - 4s 28ms/step - loss: 0.5870 - acc: 0.8706 - val_loss: 0.6474 - val_acc: 0.8628
Epoch 5/50
139/139 [==============================] - 4s 28ms/step - loss: 0.4935 - acc: 0.9204 - val_loss: 0.5573 - val_acc: 0.8881
Epoch 6/50
139/139 [==============================] - 4s 31ms/step - loss: 0.4181 - acc: 0.9267 - val_loss: 0.4827 - val_acc: 0.8881
Epoch 7/50
139/139 [==============================] - 4s 28ms/step - loss: 0.3630 - acc: 0.9294 - val_loss: 0.4272 - val_acc: 0.8881
Epoch 8/50
139/139 [==============================] - 4s 29ms/step 

In [38]:
LBBM_2_layers.evaluate(X_test, y_test)

11/11 [==============================] - 0s 15ms/step - loss: 0.0893 - acc: 0.9827


[0.08934864401817322, 0.9826589822769165]

In [39]:
LBBM_2_layers.base_model.evaluate(X_test, y_test)

11/11 [==============================] - 0s 11ms/step - loss: 0.0810 - acc: 0.9769


[0.08101316541433334, 0.9768785834312439]

In [40]:
LBBM_2_layers.unfreeze_model()
LBBM_2_layers.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005), metrics=['acc'])

In [41]:
history_LBBM_2_layers = LBBM_2_layers.fit(X_train, y_train, batch_size=8, epochs=50, verbose=1, validation_split=0.2)

Epoch 1/50
139/139 [==============================] - 94s 118ms/step - loss: 0.0444 - acc: 0.9982 - val_loss: 0.1117 - val_acc: 0.9639
Epoch 2/50
139/139 [==============================] - 5s 38ms/step - loss: 0.0428 - acc: 0.9991 - val_loss: 0.1156 - val_acc: 0.9603
Epoch 3/50
139/139 [==============================] - 5s 38ms/step - loss: 0.0416 - acc: 0.9991 - val_loss: 0.1038 - val_acc: 0.9711
Epoch 4/50
139/139 [==============================] - 5s 37ms/step - loss: 0.0411 - acc: 0.9982 - val_loss: 0.1034 - val_acc: 0.9711
Epoch 5/50
139/139 [==============================] - 5s 39ms/step - loss: 0.0397 - acc: 1.0000 - val_loss: 0.0979 - val_acc: 0.9747
Epoch 6/50
139/139 [==============================] - 6s 40ms/step - loss: 0.0432 - acc: 0.9973 - val_loss: 0.0929 - val_acc: 0.9783
Epoch 7/50
139/139 [==============================] - 5s 39ms/step - loss: 0.0382 - acc: 1.0000 - val_loss: 0.0983 - val_acc: 0.9747
Epoch 8/50
139/139 [==============================] - 6s 40ms/step 

In [42]:
LBBM_2_layers.evaluate(X_test, y_test)

11/11 [==============================] - 0s 14ms/step - loss: 0.0739 - acc: 0.9884


[0.0738501325249672, 0.9884393215179443]

In [43]:
LBBM_2_layers.base_model.evaluate(X_test, y_test)

11/11 [==============================] - 0s 9ms/step - loss: 0.0683 - acc: 0.9884


[0.06826049089431763, 0.9884393215179443]

# MultiLayerPerceptron base model:
Need to be tried with a harder task because here the MLP model quickly reache a training accuracy of 1 => Nothing more to learn from the training data.

In [38]:
MLP = MultiLayerPerceptron([tf.keras.layers.Dense(64, activation='relu'),
                           tf.keras.layers.Dense(64, activation='relu'),
                           tf.keras.layers.Dense(4, activation='softmax')])
MLP.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=['acc'])

In [39]:
history_MLP = MLP.fit(X_train, y_train, batch_size=8, epochs=50, verbose=1, validation_split=0.2)

Epoch 1/50
139/139 [==============================] - 1s 3ms/step - loss: 0.7372 - acc: 0.7131 - val_loss: 0.6040 - val_acc: 0.7112
Epoch 2/50
139/139 [==============================] - 0s 1ms/step - loss: 0.3658 - acc: 0.8507 - val_loss: 0.3740 - val_acc: 0.8303
Epoch 3/50
139/139 [==============================] - 0s 1ms/step - loss: 0.2316 - acc: 0.9041 - val_loss: 0.2766 - val_acc: 0.8809
Epoch 4/50
139/139 [==============================] - 0s 2ms/step - loss: 0.1718 - acc: 0.9421 - val_loss: 0.2258 - val_acc: 0.9025
Epoch 5/50
139/139 [==============================] - 0s 2ms/step - loss: 0.1245 - acc: 0.9656 - val_loss: 0.1739 - val_acc: 0.9386
Epoch 6/50
139/139 [==============================] - 0s 2ms/step - loss: 0.0971 - acc: 0.9756 - val_loss: 0.1429 - val_acc: 0.9422
Epoch 7/50
139/139 [==============================] - 0s 1ms/step - loss: 0.0738 - acc: 0.9855 - val_loss: 0.1239 - val_acc: 0.9675
Epoch 8/50
139/139 [==============================] - 0s 1ms/step - loss: 0.

In [40]:
MLP.evaluate(X_test, y_test)

11/11 [==============================] - 0s 2ms/step - loss: 0.0327 - acc: 0.9855


[0.03274001181125641, 0.9855491518974304]

## Retraining base model (after doing transfert learning):

In [41]:
LBBM = LorisBallsBasedModel(BoundedParaboloids(128, semi_axis_regularizer=SemiAxisRegularizer(0.00001)),
                            tf.keras.layers.Dense(4, activation='softmax'),
                            MLP)
LBBM.freeze_base_model()
LBBM.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=['acc'])

In [42]:
history_LBBM = LBBM.fit(X_train, y_train, batch_size=8, epochs=50, verbose=1, validation_split=0.2)

Epoch 1/50
139/139 [==============================] - 43s 56ms/step - loss: 1.0232 - acc: 0.7195 - val_loss: 1.0836 - val_acc: 0.6498
Epoch 2/50
139/139 [==============================] - 3s 21ms/step - loss: 0.8465 - acc: 0.7195 - val_loss: 0.9171 - val_acc: 0.6498
Epoch 3/50
139/139 [==============================] - 3s 20ms/step - loss: 0.7406 - acc: 0.7195 - val_loss: 0.7920 - val_acc: 0.6498
Epoch 4/50
139/139 [==============================] - 3s 21ms/step - loss: 0.6477 - acc: 0.7195 - val_loss: 0.7167 - val_acc: 0.6498
Epoch 5/50
139/139 [==============================] - 3s 21ms/step - loss: 0.5649 - acc: 0.7946 - val_loss: 0.6168 - val_acc: 0.8917
Epoch 6/50
139/139 [==============================] - 3s 21ms/step - loss: 0.4921 - acc: 0.9113 - val_loss: 0.5448 - val_acc: 0.8881
Epoch 7/50
139/139 [==============================] - 3s 21ms/step - loss: 0.4248 - acc: 0.9303 - val_loss: 0.4779 - val_acc: 0.8881
Epoch 8/50
139/139 [==============================] - 3s 21ms/step -

In [43]:
LBBM.evaluate(X_test, y_test)

11/11 [==============================] - 0s 7ms/step - loss: 0.0669 - acc: 0.9855


[0.06689341366291046, 0.9855491518974304]

In [44]:
LBBM.base_model.evaluate(X_test, y_test)

11/11 [==============================] - 0s 2ms/step - loss: 0.0327 - acc: 0.9855


[0.03274001181125641, 0.9855491518974304]

In [45]:
LBBM.unfreeze_model()
LBBM.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005), metrics=['acc'])

In [46]:
history_LBBM = LBBM.fit(X_train, y_train, batch_size=8, epochs=50, verbose=1, validation_split=0.2)

Epoch 1/50
139/139 [==============================] - 40s 56ms/step - loss: 0.0211 - acc: 1.0000 - val_loss: 0.0866 - val_acc: 0.9819
Epoch 2/50
139/139 [==============================] - 3s 20ms/step - loss: 0.0204 - acc: 1.0000 - val_loss: 0.0681 - val_acc: 0.9783
Epoch 3/50
139/139 [==============================] - 3s 20ms/step - loss: 0.0196 - acc: 1.0000 - val_loss: 0.0652 - val_acc: 0.9856
Epoch 4/50
139/139 [==============================] - 3s 21ms/step - loss: 0.0190 - acc: 1.0000 - val_loss: 0.0723 - val_acc: 0.9819
Epoch 5/50
139/139 [==============================] - 3s 21ms/step - loss: 0.0184 - acc: 1.0000 - val_loss: 0.0623 - val_acc: 0.9892
Epoch 6/50
139/139 [==============================] - 3s 21ms/step - loss: 0.0179 - acc: 1.0000 - val_loss: 0.0678 - val_acc: 0.9892
Epoch 7/50
139/139 [==============================] - 3s 21ms/step - loss: 0.0174 - acc: 1.0000 - val_loss: 0.0622 - val_acc: 0.9892
Epoch 8/50
139/139 [==============================] - 3s 21ms/step -

In [47]:
LBBM.evaluate(X_test, y_test)

11/11 [==============================] - 0s 9ms/step - loss: 0.0777 - acc: 0.9884


[0.07774505764245987, 0.9884393215179443]

In [48]:
LBBM.base_model.evaluate(X_test, y_test)

11/11 [==============================] - 0s 3ms/step - loss: 0.0371 - acc: 0.9884


[0.037076856940984726, 0.9884393215179443]

## Adding a second BoundedParaboloids layer

In [49]:
LBBM_2_layers = LorisBallsBasedModel(BoundedParaboloids(128, semi_axis_regularizer=SemiAxisRegularizer(0.00001)),
                                     tf.keras.layers.Dense(4, activation='softmax'),
                                     LBBM)
LBBM_2_layers.freeze_base_model()
LBBM_2_layers.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=['acc'])

In [50]:
history_LBBM_2_layers = LBBM_2_layers.fit(X_train, y_train, batch_size=8, epochs=50, verbose=1, validation_split=0.2)

Epoch 1/50
139/139 [==============================] - 72s 102ms/step - loss: 0.9842 - acc: 0.6842 - val_loss: 0.9500 - val_acc: 0.6498
Epoch 2/50
139/139 [==============================] - 4s 28ms/step - loss: 0.7693 - acc: 0.7195 - val_loss: 0.8653 - val_acc: 0.6498
Epoch 3/50
139/139 [==============================] - 4s 28ms/step - loss: 0.6746 - acc: 0.7231 - val_loss: 0.7374 - val_acc: 0.6498
Epoch 4/50
139/139 [==============================] - 4s 29ms/step - loss: 0.5856 - acc: 0.8000 - val_loss: 0.6466 - val_acc: 0.6498
Epoch 5/50
139/139 [==============================] - 4s 29ms/step - loss: 0.5048 - acc: 0.9005 - val_loss: 0.5385 - val_acc: 0.8953
Epoch 6/50
139/139 [==============================] - 4s 29ms/step - loss: 0.4196 - acc: 0.9303 - val_loss: 0.4488 - val_acc: 0.8953
Epoch 7/50
139/139 [==============================] - 4s 29ms/step - loss: 0.3456 - acc: 0.9303 - val_loss: 0.3917 - val_acc: 0.8953
Epoch 8/50
139/139 [==============================] - 4s 29ms/step 

In [51]:
LBBM_2_layers.evaluate(X_test, y_test)

11/11 [==============================] - 0s 14ms/step - loss: 0.0842 - acc: 0.9884


[0.08423987030982971, 0.9884393215179443]

In [52]:
LBBM_2_layers.base_model.evaluate(X_test, y_test)

11/11 [==============================] - 0s 6ms/step - loss: 0.0777 - acc: 0.9884


[0.07774505764245987, 0.9884393215179443]

In [53]:
LBBM_2_layers.unfreeze_model()
LBBM_2_layers.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005), metrics=['acc'])

In [54]:
history_LBBM_2_layers = LBBM_2_layers.fit(X_train, y_train, batch_size=8, epochs=50, verbose=1, validation_split=0.2)

Epoch 1/50
139/139 [==============================] - 117s 355ms/step - loss: 0.0298 - acc: 1.0000 - val_loss: 0.0669 - val_acc: 0.9928
Epoch 2/50
139/139 [==============================] - 5s 38ms/step - loss: 0.0289 - acc: 1.0000 - val_loss: 0.0613 - val_acc: 0.9928
Epoch 3/50
139/139 [==============================] - 5s 39ms/step - loss: 0.0280 - acc: 1.0000 - val_loss: 0.0613 - val_acc: 0.9928
Epoch 4/50
139/139 [==============================] - 6s 40ms/step - loss: 0.0272 - acc: 1.0000 - val_loss: 0.0628 - val_acc: 0.9928
Epoch 5/50
139/139 [==============================] - 6s 40ms/step - loss: 0.0265 - acc: 1.0000 - val_loss: 0.0625 - val_acc: 0.9928
Epoch 6/50
139/139 [==============================] - 6s 45ms/step - loss: 0.0259 - acc: 1.0000 - val_loss: 0.0590 - val_acc: 0.9928
Epoch 7/50
139/139 [==============================] - 7s 53ms/step - loss: 0.0252 - acc: 1.0000 - val_loss: 0.0671 - val_acc: 0.9892
Epoch 8/50
139/139 [==============================] - 6s 41ms/step

In [55]:
LBBM_2_layers.evaluate(X_test, y_test)

11/11 [==============================] - 0s 15ms/step - loss: 0.0938 - acc: 0.9884


[0.09377098083496094, 0.9884393215179443]

In [56]:
LBBM_2_layers.base_model.evaluate(X_test, y_test)

11/11 [==============================] - 0s 8ms/step - loss: 0.0751 - acc: 0.9884


[0.07514117658138275, 0.9884393215179443]